In [7]:
import pandas as pd

In [8]:
df_train_raw_data = pd.read_csv(r"C:\Users\HP\Downloads\train - train.csv")
df_item_raw_data = pd.read_csv(r"C:\Users\HP\Downloads\item_data - item_data.csv")
df_ct_raw_data = pd.read_csv(r"C:\Users\HP\Downloads\customer_transaction_data.csv")
df_cd_raw_data = pd.read_csv(r"C:\Users\HP\Downloads\customer_demographics - customer_demographics.csv")
df_cim_raw_data = pd.read_csv(r"C:\Users\HP\Downloads\coupon_item_mapping - coupon_item_mapping.csv")
df_campaign_raw_data = pd.read_csv(r"C:\Users\HP\Downloads\campaign_data - campaign_data.csv")

In [9]:
def get_df_info(df, include_unique_values=False):
    col_name_list = list(df.columns)
    col_type_list = [type(df[col][0]) for col in col_name_list]
    col_null_count_list = [df[col].isnull().sum() for col in col_name_list]
    col_unique_count_list = [df[col].nunique() for col in col_name_list]
    col_memory_usage_list = [df[col].memory_usage(deep=True) for col in col_name_list]
    df_total_memory_usage = sum(col_memory_usage_list) / 1048576
    if include_unique_values:
        col_unique_list = [df[col].unique() for col in col_name_list]
        df_info = pd.DataFrame({'column_name': col_name_list, 'type': col_type_list, 
                                'null_count': col_null_count_list, 'nunique': col_unique_count_list, 
                                'unique_values': col_unique_list})
    else:
        df_info = pd.DataFrame({'column_name': col_name_list, 'type': col_type_list, 
                                'null_count': col_null_count_list, 'nunique': col_unique_count_list})
    return df_info, df_total_memory_usage

In [10]:
df_train_raw_data.head()

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0
3,7,13,644,1050,0
4,9,8,1017,1489,0


In [11]:
df_train_raw_data_info, df_train_raw_data_mem = get_df_info(df_train_raw_data, True)
print(f'train dataset has {df_train_raw_data.shape[0]} rows and {df_train_raw_data.shape[1]} cols, uses approx. {df_train_raw_data_mem:.2f} MB')
df_train_raw_data_info

train dataset has 78369 rows and 5 cols, uses approx. 2.99 MB


,column_name,type,null_count,nunique,unique_values
0,id,<class 'numpy.int64'>,0,78369,"[1, 2, 6, 7, 9, 11, 14, 15, 17, 19, 20, 21, 22..."
1,campaign_id,<class 'numpy.int64'>,0,18,"[13, 9, 8, 11, 29, 30, 2, 5, 12, 26, 3, 4, 10,..."
2,coupon_id,<class 'numpy.int64'>,0,866,"[27, 116, 635, 644, 1017, 795, 444, 538, 857, ..."
3,customer_id,<class 'numpy.int64'>,0,1428,"[1053, 48, 205, 1050, 1489, 793, 590, 368, 523..."
4,redemption_status,<class 'numpy.int64'>,0,2,"[0, 1]"


In [12]:
len(set(df_train_raw_data['customer_id']).intersection(set(df_cd_raw_data['customer_id'])))

703

In [13]:
df_coupon_item_merged = df_cim_raw_data.merge(df_item_raw_data, on='item_id')

In [14]:
df_coupon_item_merged.head()

,coupon_id,item_id,brand,brand_type,category
0,105,37,56,Local,Grocery
1,6,37,56,Local,Grocery
2,22,37,56,Local,Grocery
3,31,37,56,Local,Grocery
4,107,75,56,Local,Grocery


In [15]:
df_coupon_item_merged_info, df_coupon_item_merged_mem = get_df_info(df_coupon_item_merged, True)
print(f'coupon_item_merged dataset has {df_coupon_item_merged.shape[0]} rows and {df_coupon_item_merged.shape[1]} cols, uses approx. {df_coupon_item_merged_mem:.2f} MB')
df_coupon_item_merged_info

coupon_item_merged dataset has 92663 rows and 5 cols, uses approx. 27.65 MB


,column_name,type,null_count,nunique,unique_values
0,coupon_id,<class 'numpy.int64'>,0,1116,"[105, 6, 22, 31, 107, 9, 21, 30, 226, 494, 492..."
1,item_id,<class 'numpy.int64'>,0,36289,"[37, 75, 76, 77, 81, 90, 98, 101, 105, 111, 11..."
2,brand,<class 'numpy.int64'>,0,2555,"[56, 209, 278, 487, 172, 634, 89, 676, 1016, 1..."
3,brand_type,<class 'str'>,0,2,"[Local, Established]"
4,category,<class 'str'>,0,17,"[Grocery, Bakery, Skin & Hair Care, Pharmaceut..."


In [16]:
df_cim_gb_cid = df_coupon_item_merged.groupby('coupon_id')
df_train_merged = df_train_raw_data.copy(deep=True)

In [17]:
df_train_merged['coupon_items_count'] = df_train_merged.apply(lambda x: (df_cim_gb_cid.get_group(x['coupon_id'])['item_id']).count(), axis=1)
df_train_merged['coupon_items_unique_brands'] = df_train_merged.apply(lambda x: (df_cim_gb_cid.get_group(x['coupon_id'])['brand']).nunique(), axis=1)

In [18]:
df_train_merged.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,coupon_items_count,coupon_items_unique_brands
0,1,13,27,1053,0,125,2
1,2,13,116,48,0,3,1
2,6,9,635,205,0,67,1
3,7,13,644,1050,0,4,1
4,9,8,1017,1489,0,32,1


In [19]:
df_train_merged_info, df_train_merged_mem = get_df_info(df_train_merged, True)
print(f'train_merged dataset has {df_train_merged.shape[0]} rows and {df_train_merged.shape[1]} cols, uses approx. {df_train_merged_mem:.2f} MB')
df_train_merged_info

train_merged dataset has 78369 rows and 7 cols, uses approx. 4.19 MB


,column_name,type,null_count,nunique,unique_values
0,id,<class 'numpy.int64'>,0,78369,"[1, 2, 6, 7, 9, 11, 14, 15, 17, 19, 20, 21, 22..."
1,campaign_id,<class 'numpy.int64'>,0,18,"[13, 9, 8, 11, 29, 30, 2, 5, 12, 26, 3, 4, 10,..."
2,coupon_id,<class 'numpy.int64'>,0,866,"[27, 116, 635, 644, 1017, 795, 444, 538, 857, ..."
3,customer_id,<class 'numpy.int64'>,0,1428,"[1053, 48, 205, 1050, 1489, 793, 590, 368, 523..."
4,redemption_status,<class 'numpy.int64'>,0,2,"[0, 1]"
5,coupon_items_count,<class 'numpy.int64'>,0,104,"[125, 3, 67, 4, 32, 13, 14, 9, 31, 6, 147, 134..."
6,coupon_items_unique_brands,<class 'numpy.int64'>,0,14,"[2, 1, 955, 224, 1187, 371, 3, 170, 5, 128, 52..."
